Follow the steps below to learn how to use R and lpSolveAPI library to solve linear programming models.

Analysed problem:

A calculator company produces a scientific calculator and a graphing calculator. Long-term projections indicate an expected demand of at least 100 scientific and 80 graphing calculators each day. Because of limitations on production capacity, no more than 200 scientific and 170 graphing calculators can be made daily. To satisfy a shipping contract, a total of at least 200 calculators much be shipped each day.
If each scientific calculator sold results in a `$`2 loss, but each graphing calculator produces a `$`5 profit, how many of each type should be made daily to maximize net profits?

0) Install lpSolveAPI package if needed. 

In [1]:
#install.packages("lpSolveAPI")

1) Load necessary package

In [2]:
library(lpSolveAPI)

2) Load data 

Load data containing profits and limits from csv file. Data about predicted profit, demand and maximal production of each type of calculator are saved in calculators.csv file.

Use read.csv to read these files. Remenber that your files have headers and fields which are separated with semicolons. 
https://stat.ethz.ch/R-manual/R-devel/library/utils/html/read.table.html

In [3]:
data <- read.csv("calculators.csv", header = TRUE, sep = ";")
data

X,scientific,graphic
<fct>,<int>,<int>
profit,-2,5
demand,100,80
max prodution,200,170


3) Firstly model the problem. We introduce 2 variables s and g meaning, respectively, the number of scientific and the number of graphic calculators. We want to maximize company's profit:

\begin{equation*}
max \quad -2s+5g
\end{equation*}

Consider the following constraints:
- minimal demand for each type of calculator:
\begin{equation*}
s >= 100 \\
g >= 80
\end{equation*}

- production limit for each type of calculator:
\begin{equation*}
s <= 200 \\
g <= 170
\end{equation*}

- minimal total number of calculators:
\begin{equation*}
s+g >= 200
\end{equation*}

4) Create LP model object using make.lp function.
https://www.rdocumentation.org/packages/lpSolveAPI/versions/5.5.2.0-17/topics/make.lp

The only data you need to pass to this function is a number of decision variables (columns) in your problem.

In [4]:
model <- make.lp(ncol = ncol(data) - 1)

5) Set model's optimization direction and variable types - the number of calculators is an integer variable!
To set the direction use lp.control function. Look in a documentation for the parameters which have to be passed.
https://www.rdocumentation.org/packages/lpSolveAPI/versions/5.5.2.0-17/topics/lp.control
For variable types use set.type functionpassing your model, indexes of variables to be set (in your case: an array containing values 1 and 2), and type (in your case "integer").

In [5]:
lp.control(model, sense = "max")
set.type(model, c(1, 2), type = "integer")

$anti.degen
[1] "fixedvars" "stalling" 

$basis.crash
[1] "none"

$bb.depthlimit
[1] -50

$bb.floorfirst
[1] "automatic"

$bb.rule
[1] "pseudononint" "greedy"       "dynamic"      "rcostfixing" 

$break.at.first
[1] FALSE

$break.at.value
[1] 1e+30

$epsilon
      epsb       epsd      epsel     epsint epsperturb   epspivot 
     1e-10      1e-09      1e-12      1e-07      1e-05      2e-07 

$improve
[1] "dualfeas" "thetagap"

$infinite
[1] 1e+30

$maxpivot
[1] 250

$mip.gap
absolute relative 
   1e-11    1e-11 

$negrange
[1] -1e+06

$obj.in.basis
[1] TRUE

$pivoting
[1] "devex"    "adaptive"

$presolve
[1] "none"

$scalelimit
[1] 5

$scaling
[1] "geometric"   "equilibrate" "integers"   

$sense
[1] "maximize"

$simplextype
[1] "dual"   "primal"

$timeout
[1] 0

$verbose
[1] "neutral"

6) To add an objective function to your model, pass a vector of coefficiencs (one coefficient for one decision variable). For this reason, read profit values from data variable (read in step 2). You can use matrix slices in R. For example if you want to choose second row and columns 2-5 from matrix X you can use the following slice X[2,2:5]. After choosing proper coefficients, use set.objfn function to set them as objective in your model.

In [6]:
objective_coeffs <- data[1, c(2,3)]
objective_coeffs
set.objfn(model, objective_coeffs)

scientific,graphic
<int>,<int>
-2,5


7) Add model constraints. Use again data variable to get minimal demand and maximal production for each type of calculator. *Remember to add an additional constraint imposing a minimal number of calculators to at least 200.*

To set constraints, use add.constraint function passing your model, constraint coefficients (if some of variables are not present in a given costraint, set coefficient for this variable to 0), constraint type ("<=", ">=" or "=") and rhs.

In [7]:
constraint_coeffs <- c(1, 0)
add.constraint(model, constraint_coeffs, type = ">=", rhs = data[2, 2]) #minimal number of scientific
add.constraint(model, constraint_coeffs, type = "<=", rhs = data[3, 2]) #maximal number of scientific

constraint_coeffs <- c(0, 1)
add.constraint(model, constraint_coeffs, type = ">=", rhs = data[2, 3]) #minimal number of graphic
add.constraint(model, constraint_coeffs, type = "<=", rhs = data[3, 3]) #maximal number of graphic

constraint_coeffs <- array(1, dim = 2)
add.constraint(model, constraint_coeffs, type = ">=", rhs = 200 ) #minimal total number of calculators

8) Run the solver. Use solve function

In [8]:
solve(model)

[1] 0

9) Display maximal profit and number of calculators wchich have to be produced to maximize the profit. Use get.objective and get.variables functions.

In [9]:
get.objective(model)
get.variables(model)

[1] 650

[1] 100 170